In [104]:
import math
import cmath
import altair as alt
import numpy as np
import scipy 
import pandas as pd

In [105]:
names = ['Freq', 'dB', 'Phase']
speaker = 'Genelec 8361 + W371 GLM (ON)'
fs = 48000
left = pd.read_csv("../datas/inroom/m002/L.large.txt", skiprows=14, names=names)
right = pd.read_csv("../datas/inroom/m002/R.large.txt", skiprows=14, names=names)

In [106]:
def spl2pressure(spl):
    return 10**((spl-105)/20)

def pressure2spl(p):
    return 105.0 + 20.0 * np.log10(p)

In [115]:
def expend(df):
    nFFT = len(df.Freq)+1
    # extend freq from 0 to fs/2
    freq = np.empty(len(df.Freq.values)+2)
    freq[0] = 0
    freq[1:-1]= df.Freq.values
    freq[-1] = fs/2
    # interpolate to have a gain at 0 Hz
    gain_0 = df.dB[0]-df.Freq[0]*(df.dB[1]-df.dB[0])/(df.Freq[1]-df.Freq[0])
    # interpolate for have a gain at fs/2
    gain_fs2 = df.dB.values[-1]/2
    gain = np.empty(len(df.Freq.values)+2)
    gain[0] = gain_0
    gain[1:-1] = df.dB.values
    gain[-1] = gain_fs2
    # interpolate across freq range
    tck = interpolate.splrep(freq, gain)
    gain = interpolate.splev(np.linspace(0, fs/2, nFFT), tck)
    # symmetrize output
    sym_gain = np.empty(2*nFFT-1)
    sym_gain[0:nFFT] = gain
    sym_gain[nFFT:] = gain[1:]
    return sym_gain

def minimum_phase(df):
    x = expend(df)
    n = len(x)
    ceps = np.fft.ifft(x)
    ceps_folded
    ifft = np.fft.fft(ceps_folded)
    min_phase = ifft[0:len(ifft)//2]
    return pressure2spl(min_phase)

left['MinPhase'] = minimum_phase(left)
left['ExcessPhase'] = np.angle( np.array([cmath.rect(1, l*math.pi/180) for l in left.Phase])
                               -np.array([cmath.rect(1, m*math.pi/180) for m in left.MinPhase]), 
                               deg=True)
right['MinPhase'] = minimum_phase(right)
right['ExcessPhase'] = np.angle( np.array([cmath.rect(1, l*math.pi/180) for l in right.Phase])
                                -np.array([cmath.rect(1, m*math.pi/180) for m in right.MinPhase]), 
                                deg=True)

<ipython-input-106-3ada9d983802>:5: RuntimeWarning: invalid value encountered in log10
  return 105.0 + 20.0 * np.log10(p)


In [116]:
def group_delay(freq, phase):
    gd = np.zeros_like(freq)
    gd[0] = -1*(phase[1]-phase[0])/(freq[1]-freq[0])/360
    gd[1:-1] = [((phase[n]-phase[n-1])/(freq[n]-freq[n-1]))+((phase[n+1]-phase[n])/(freq[n+1]-freq[n])) for n in range(1, len(freq)-1)]
    gd[1:-1] = -gd[1:-1]/720
    gd[-1] = -1*(phase[-1]-phase[-2])/(freq[-1]-freq[-2])/360
    return gd

left['GroupDelay'] = group_delay(left.Freq.values, np.unwrap(left.Phase.values, 180))
right['GroupDelay'] = group_delay(right.Freq.values, np.unwrap(right.Phase.values, 180))

left['MinGroupDelay'] = group_delay(left.Freq.values, np.unwrap(left.MinPhase.values, 180))
right['MinGroupDelay'] = group_delay(right.Freq.values, np.unwrap(right.MinPhase.values, 180))

left['ExcessGroupDelay'] = group_delay(left.Freq.values, np.unwrap(left.ExcessPhase.values, 180))
right['ExcessGroupDelay'] = group_delay(right.Freq.values, np.unwrap(right.ExcessPhase.values, 180))

/home/pierre/src/spinorama/spinorama-venv/lib/python3.8/site-packages/numpy/lib/function_base.py:1540: RuntimeWarning: invalid value encountered in remainder
  ddmod = mod(dd + pi, 2*pi) - pi


In [117]:
def impulse_response(df):
    ir = np.fft.ifft(spl2pressure(expend(df))).real
    return ir[0, len(ir)//2]
    
left_z = impulse_response(left)    

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [118]:
def plot(df, f_max):
    x_axis=alt.X('Freq:Q', title='Frequency (Hz)', axis=alt.Axis(format='~s'), scale=alt.Scale(type="log", nice=False, domain=[20, f_max]))
    freq = alt.Chart(df.loc[df.Freq<=f_max]).mark_line(
    ).encode(
        x=x_axis,
        y=alt.Y('dB:Q', title='Sound Pressure (dB SPL)', scale=alt.Scale(zero=False, domain=[30, 80])),
        color=alt.value('blue')
    ).properties(width=500, height=300)
    
    isoTicks = [-180 + 30 * i for i in range(0, 13)]
    phase = alt.Chart(df.loc[df.Freq<=f_max]).mark_line(
        clip=True,
    ).encode(
        x=x_axis,
        y=alt.Y('Phase:Q', 
                axis=alt.Axis(title='Phase (deg)', values=isoTicks, labelExpr="datum.value+'°'"), 
                scale=alt.Scale(zero=False, nice=False, domain=[-180, 180])),
        color=alt.value('blue')
    ).properties(width=500, height=200)
    min_phase = alt.Chart(df.loc[df.Freq<=f_max]).mark_line(
        clip=True
    ).encode(
        x=x_axis,
        y=alt.Y('MinPhase:Q', title='Phase (deg)', scale=alt.Scale(zero=False, nice=False, domain=[-180, 180])),
        color=alt.value('red')
    ).properties(width=500, height=200)
    excess_phase = alt.Chart(df.loc[df.Freq<=f_max]
    ).mark_line(
        clip=True
    ).encode(
        x=x_axis,
        y=alt.Y('ExcessPhase:Q', title='Phase (deg)', scale=alt.Scale(zero=False, nice=False, domain=[-180, 180])),
        color=alt.value('green')
    ).properties(width=500, height=200)
    
    group_delay = alt.Chart(df.loc[df.Freq<=f_max]).mark_line(
        clip=True
    ).encode(
        x=x_axis,
        y=alt.Y('GroupDelay:Q', 
                axis=alt.Axis(title='Group Delay (ms)'), 
                scale=alt.Scale(zero=False, nice=False, domain=[-0.2, 0.2])),
        color=alt.value('blue')
    ).properties(width=500, height=200)
    min_group_delay = alt.Chart(df.loc[df.Freq<=f_max]).mark_line(
        clip=True
    ).encode(
        x=x_axis,
        y=alt.Y('MinGroupDelay:Q', title='Group Delay (ms)', scale=alt.Scale(zero=False, nice=False, domain=[-0.2, 0.2])),
        color=alt.value('red')
    ).properties(width=500, height=200)
    excess_group_delay = alt.Chart(df.loc[df.Freq<=f_max]).mark_line(
        clip=True
    ).encode(
        x=x_axis,
        y=alt.Y('ExcessGroupDelay:Q', title='Group Delay (ms)', scale=alt.Scale(zero=False, nice=False)),
        color=alt.value('green')
    ).properties(width=500, height=200)
    chart = alt.vconcat(freq, phase+min_phase+excess_phase, group_delay+min_group_delay+excess_group_delay)
    return chart

In [119]:
f_max = 200
alt.hconcat(
    plot(left, f_max).properties(title='{} Left'.format(speaker)), 
    plot(right, f_max).properties(title='{} Right'.format(speaker)))

alt.HConcatChart(...)

In [91]:
nFFT = max(512, len(freq))
# extend freq from 0 to fs/2
freq = np.empty(len(df.Freq.values)+2)
freq[0] = 0
freq[1:-1]= df.Freq.values
freq[-1] = fs/2
# interpolate to have a gain at 0 Hz
gain_0 = df.dB[0]-df.Freq[0]*(df.dB[1]-df.dB[0])/(df.Freq[1]-df.Freq[0])
# interpolate for have a gain at fs/2
gain_fs2 = df.dB.values[-1]/2
gain = np.empty(len(df.Freq.values)+2)
gain[0] = gain_0
gain[1:-1] = df.dB.values
gain[-1] = gain_fs2
tck = interpolate.splrep(freq, gain)
gain = interpolate.splev(np.linspace(0, fs/2, nFFT), tck)

In [92]:
sym_gain = np.empty(2*nFFT-1)
sym_gain[0:nFFT] = gain
sym_gain[nFFT:] = gain[1:]

In [98]:
impulse_resp = np.fft.ifft(spl2pressure(expend(right))).real[0:nFFT-2]

<ipython-input-58-3ada9d983802>:2: RuntimeWarning: overflow encountered in power
  return 10**((spl-105)/20)


In [99]:
alt.data_transformers.disable_max_rows()

nb_samp = 100
alt.Chart(pd.DataFrame(
    {'t': range(0, nb_samp), 
     'ir': impulse_resp[0:nb_samp]
    })).mark_line(
).encode(
    x='t', 
    y='ir:Q'
)

alt.Chart(...)